In [40]:
import openai
import pandas as pd

In [41]:
client = openai.Client()

In [42]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose="assistants"
)

In [43]:
print(file.id)

file-9fcanCAUP1aimn5QsTrFeo


In [44]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é uma analista que analisa dados sobre vendas",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids":[file.id]}},
    model="gpt-4o",
)

In [45]:
# pergunta = "Qual o rating médio das vendas do supermercado?"
pergunta = "Quantas colunas tem o arquivo e quais saão elas?"

In [46]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\832815408.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\832815408.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [47]:
# Executa a thread 
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\209825715.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [48]:
# Aguarda a thread rodar
import time

while run.status in ["queued", "in_progress", "cancellig"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\435338251.py:6: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [49]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\184256488.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_uRTGaUgeBhV1jLPSK4FH6Ouq', assistant_id='asst_pcKins3M0chja8LuWFDnsdwt', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O arquivo possui 17 colunas. Elas são:\n\n1. Invoice ID\n2. Branch\n3. City\n4. Customer type\n5. Gender\n6. Product line\n7. Unit price\n8. Quantity\n9. Tax 5%\n10. Total\n11. Date\n12. Time\n13. Payment\n14. cogs\n15. gross margin percentage\n16. gross income\n17. Rating'), type='text')], created_at=1749766334, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_pDRYgOkmIPYj3QrnNv86U1tG', status=None, thread_id='thread_496h4T97tGTZce3idqyy4cAW'), Message(id='msg_LRxTXxiOu88Bq5WPk2YP3MbR', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Quantas colunas tem o arquivo e quais saão elas?'), type='text')], created_at=1749766323, incomplete_at=None, incompl

In [50]:
# mensagens.data[0].content[0].text.value

In [51]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\1597243276.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [52]:
for step in run_steps.data[::-1]:
    print(f"\n=====Step {step.step_details.type}")

    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" * 10)
            print(tool_call.code_interpreter.input)
            print("=" * 10)

    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )

        if message.content[0].type == "text":
            print(message.content[0].text.value)
        
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)

            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva.")


        # print(message.content[0].text.value)


=====Step tool_calls
# Vamos primeiro importar as bibliotecas necessárias para carregar e inspecionar o arquivo.
import pandas as pd

# Carregar o arquivo para visualizar as primeiras linhas e entender sua estrutura.
file_path = '/mnt/data/file-9fcanCAUP1aimn5QsTrFeo'
data = pd.read_csv(file_path)

# Obter o número de colunas e seus nomes
num_columns = data.shape[1]
column_names = data.columns.tolist()

num_columns, column_names

=====Step message_creation


C:\Users\kaues\AppData\Local\Temp\ipykernel_16244\1695954454.py:11: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


O arquivo possui 17 colunas. Elas são:

1. Invoice ID
2. Branch
3. City
4. Customer type
5. Gender
6. Product line
7. Unit price
8. Quantity
9. Tax 5%
10. Total
11. Date
12. Time
13. Payment
14. cogs
15. gross margin percentage
16. gross income
17. Rating
